In [ ]:
import pandas as pd
import numpy as np
import datetime

## 要输入的地方是lotid，读取表格，如果有 CancelTrackIn 的要读两遍

In [ ]:
L = 'L2E9307A8AA'
dfm = pd.read_excel('./%s/M_%s.xls' % (L, L))
dfcm = pd.read_excel('./%s/MC_%s.xls' % (L, L))
dfo = pd.read_excel('./%s/O_%s.xls' % (L, L))
print(dfm.shape)
print(dfcm.shape)
print(dfo.shape)

## 这一步是找到有没有canceltrackin，如果有要删掉对应的trackin才行,如果执行了删除操作，就需要再读取一遍文件，也就是再run上一个cell，观察dfo的行数有没有变化

In [ ]:
dfo[dfo['EventName'] == 'CancelTrackIn']

## 全部的处理过程

In [ ]:
Lotid = dfm['LOTID'][0]

dfcm['TIME'] = pd.to_datetime(dfcm['TIME'])
AUU_start = dfcm[(dfcm['DEST MACHINE'] == '2AUU01') & (dfcm['STATE'] == 'CREATED')]['TIME']
print('%s 进入Array时间为 %s 。' % (Lotid ,AUU_start.iloc[0]))

dfm = dfm[['TIME', 'TRANSPORTJOBID', 'STATE']]  #只选取我们需要用到的三列'TIME', 'TRANSPORTJOBID', 'STATE'
print(dfm.shape)
l = dfm['TRANSPORTJOBID']
dfmcount = pd.DataFrame([l.value_counts()]).T  # 统计每个TRANSPORTJOBID出现的次数
list_one = dfmcount[dfmcount['TRANSPORTJOBID'] == 1].index.tolist()  # 如果出现次数是1，记录下来下一步准备删除
print(len(list_one))
dfm=dfm[-dfm.TRANSPORTJOBID.isin(list_one)]  # 删除TRANSPORTJOBID是唯一的记录
print(dfm.shape)

dfm_time = pd.DataFrame()  # 新建一个表格用来处理搬送时间
# 原表中‘STATE’是‘CREATED’的那些行的时间提取出来
dfm_time['created'] = dfm[dfm['STATE'] == 'CREATED']['TIME'].tolist()
# 原表中‘STATE’不是‘CREATED’（也就是‘COMPLETED’或者是‘TERMINATED’）的那些行的时间提取出来
dfm_time['ended'] = dfm[dfm['STATE'] != 'CREATED']['TIME'].tolist()
# 转换成时间格式，为下一步计算差值以及总时间做准备
dfm_time = dfm_time.astype(np.datetime64)
# 计算每次搬送的时间，并求和
dfm_time['delta'] = dfm_time['ended'] - dfm_time['created']
trantime = dfm_time['delta'].sum()
print('%s 在 Array 搬送总时间为： %s' % (Lotid, trantime))

dfo = dfo[['EventTime', 'EventName', 'EQP ID']]                 # 只提取'EventTime', 'EventName'后续处理
dfo['EventTime'] = pd.to_datetime(dfo['EventTime'])   # 转换时间戳，便于后面的计算
shipidx = int(dfo[dfo['EventName'] == 'Ship'].index.values)
dfo = dfo.iloc[shipidx:]
ship_time = dfo[dfo['EventName'] == 'Ship']['EventTime']
sumtime = (ship_time.iloc[0] - AUU_start.iloc[0])
print('%s 在 Array 总的时间为：%s ' % (Lotid, sumtime))

dfo_time = pd.DataFrame()
dfo_time['tki'] = dfo[dfo['EventName'] == 'TrackIn']['EventTime'].tolist()
dfo_time['tko'] = dfo[dfo['EventName'] == 'TrackOut']['EventTime'].tolist()
dfo_time = dfo_time.astype(np.datetime64)
dfo_time['delta'] = dfo_time['tko'] - dfo_time['tki']
runtime = dfo_time['delta'].sum()
print('%s 在 Array 总的制程时间为 ： %s' % (Lotid, runtime))

dfo_time = pd.DataFrame()
dfo_time['tki'] = dfo[dfo['EventName'] == 'TrackIn']['EventTime'].tolist()
dfo_time['tko'] = dfo[dfo['EventName'] == 'TrackOut']['EventTime'].tolist()
dfo_time = dfo_time.astype(np.datetime64)
dfo_time['delta'] = dfo_time['tko'] - dfo_time['tki']
runtime = dfo_time['delta'].sum()
print('%s 在 Array 总的制程时间为 ： %s' % (Lotid, runtime))

waittime = (sumtime - trantime - runtime)
print('%s 在 Array 总的等待时间为：% s' % (Lotid, waittime))

## 下面格式输出，稍微美观一点

In [ ]:
print('本批lot为： %s ' % Lotid)
print('%s 在 Array 总的时间为:       %s' % (Lotid, sumtime))
print('%s 在 Array 搬送总时间为:     %s' % (Lotid, trantime))
print('%s 在 Array 总的制程时间为:   %s' % (Lotid, runtime))
print('%s 在 Array 总的等待时间为:   %s' % (Lotid, waittime))

## 转换成秒，绘制饼图

In [ ]:
s_runtime = runtime.total_seconds()
s_trantime = trantime.total_seconds()
s_waittime = waittime.total_seconds()
# print(s_runtime, s_trantime, s_waittime)

from pyecharts import Pie

attr = ["制程时间", "搬送时间", "等待时间"]
s_time = [s_runtime, s_trantime, s_waittime]
pie = Pie(('%s在Array总时长%s' % (Lotid, sumtime)), width=900, height=500,  title_pos='center', title_text_size=21)
pie.add("", attr, s_time, 
        is_label_show=True,
        radius=[45, 70], 
        legend_pos = 'right',
        legend_orient = 'vertical',
        label_text_size=19, legend_text_size=17)